# Python, Data, 2025

---

* [Manipulace se sloupci](#Spojov%C3%A1n%C3%AD-dat),
* [Spojování dat](#Spojov%C3%A1n%C3%AD-dat),
    - [spojování s CONCAT](#Spojování-s-CONCAT),
    - [spojování s CONCAT a JOIN](#concat-a-volitelný-argument-join),
    - [spojování s metodou APPEND](#Spojování-pomocí-metody-APPEND),
    - [spojování s MERGE](#Spojování-pomocí-merge),
    - [spojování s metodou JOIN](#Spojení-pomocí-indexů,-join),
* [Agregace](#Agregace),
    - [jednoduchá agregace](#Jednoduchá-agregace),
    - [seskupování GROUPBY](#Seskupování-groupby),
    - [agregace](#Agregace-s-metodami),
    - [filtrování](#Filtrování),
    - [transformace](#Transformace),
    - [apply](#Metoda-apply),
* [Pivot tabulka](#Pivot-tabulky),
    - [úvodní motivace](),
    - [syntaxe tabulky](),
    - [doplňující možnosti]().
---

<br>

## Manipulace se sloupci

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse4.mm.bing.net%2Fth%3Fid%3DOIP.tzaGc-M5pl0DP6pGu-H6WgHaHa%26pid%3DApi&f=1&ipt=39909793c49bf980b8dbceaabe312870e1446f58fba1e6b430630cfbddd2b9c4&ipo=images" width="200" style="margin-left:auto; margin-right:auto"/>



Jakmile se dovedeš zorientovat v datových strukturách knihovny `pandas`, můžeš začít upravovat jejich obsah:
1. **Přejmenovat** sloupce,
2. **vytvořit** nové sloupce,
3. **odstranit** existující sloupce.

<img src="https://imgs.search.brave.com/gKVKM0gMPdcdyI_mt5KyWfGrx-nGAFVdDOrSLlHo5wo/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9pbWFn/ZXM3Lm1lbWVkcm9p/ZC5jb20vaW1hZ2Vz/L1VQTE9BREVEODAw/LzY0ZDE1NGQ4ZGQy/MTMuanBlZw" width="300" style="margin-left:auto; margin-right:auto"/>

### Přejmenování sloupců

---

Pokud budeš někdy zápasit se špatně pojmenovaným sloupcem nebo sloupci, přejmenuj je pomocí metody `rename`.

In [1]:
from pandas import DataFrame

In [2]:
uzivatele = {'jmeno': ['Matouš', 'Marek', 'Lukáš', 'Jan'],
             'age': [23, 25, 27, 29],
             'vaha': [65, 70, 75, 80],
             'vyska': [160, 170, 180, 190]}

In [3]:
uzivatele_df = DataFrame(uzivatele)

In [4]:
uzivatele_df

,jmeno,age,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


V ukázce výš tě omezuje **jazyková nekonzistence** mezi sloupečky.

Pomocí metody `rename` a argumentu `columns`, můžeš pomocí `dict` anotace vybrat, co a čím přejmenovat:

In [5]:
uzivatele_df.rename(columns={"age": "vek"})

,jmeno,vek,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


Při další kontrole zjistíš, že **nové jméno sloupečku zmizelo**:

In [6]:
uzivatele_df

,jmeno,age,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


<br>

Celou situaci můžeš řešit **dvěma způsoby**:
1. Volitelný argument `inplace`,
2. kopie původního `DataFrame` objektu.

In [7]:
prejmenovany_sloupec_df = uzivatele_df.rename(columns={"age": "vek"})

In [8]:
prejmenovany_sloupec_df

,jmeno,vek,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


In [9]:
uzivatele_df

,jmeno,age,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


In [10]:
uzivatele_df.rename(columns={"age": "vek"}, inplace=True)  # ⚠️ Za cenu rizika!⚠️

In [11]:
uzivatele_df

,jmeno,vek,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


Opatrně na práci s argumentem `inplace`.

Jeho použití vždy důkladně zvaž, ať neztratíš zdrojová data.

<br>

Přejmenovat můžeš i **více sloupců** současně:

In [12]:
# CZ --> EN
nova_jmena_sloupcu = {"jmeno": "first_name",
                      "vek": "age",
                      "vaha": "weight",
                      "vyska": "height"}

In [13]:
uzivatele_df.rename(columns=nova_jmena_sloupcu, inplace=True) # ⚠️

In [14]:
uzivatele_df

,first_name,age,weight,height
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


<br>

Občas vhodnější řešení je provést transformaci hodnot a přidání nového sloupce.



<img src="https://imgs.search.brave.com/Q3iNWCeKLnizjFbTrtaUEiEcXWvAqdNi_cWO1vwkJo0/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9pbWFn/ZXMuaWNvbi1pY29u/cy5jb20vMTE1NC9Q/TkcvOTYvMTQ4NjU2/NDQwNy1wbHVzLWdy/ZWVuXzgxNTIxLnBu/Zw" width="100" style="margin-left:auto; margin-right:auto"/>

### Přidávání sloupců do DataFrame

---

Potřebuješ doplnit další sloupec. Nebo upravit data v jednom sloupci a výsledek zapsat do dalšího.

Nejčastěji můžeš pracovat **na přidávání nových sloupců** těmito třemi způsoby:
1. **Slovníková anotace** pomocí klíče a hodnoty,
2. výsledek pomocí operace **mezi dvěma sloupci**,
3. pomocí metody `apply`.

#### Jako klíč a hodnota

---

Takový způsob použiješ, pokud máš nachystané hodnoty pro nový sloupec:

In [15]:
# EN --> CZ
nova_jmena_sloupcu = {"first_name": "jmeno",
                      "age": "vek",
                      "weight": "vaha",
                      "height": "vyska"}

In [16]:
uzivatele_df.rename(columns=nova_jmena_sloupcu, inplace=True)

In [17]:
uzivatele_df

,jmeno,vek,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


<br>

Pokud přiřazuji existující hodnoty, pomocí **slovníkové anotace**, skládám hodnoty podle pořadí:

In [18]:
uzivatele_df['prijmeni'] = ("Holinka", "Párek", "Holinka", "Novák")

In [19]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni
0,Matouš,23,65,160,Holinka
1,Marek,25,70,170,Párek
2,Lukáš,27,75,180,Holinka
3,Jan,29,80,190,Novák


In [20]:
uzivatele_df.iloc[1]

jmeno       Marek
vek            25
vaha           70
vyska         170
prijmeni    Párek
Name: 1, dtype: object

<br>

#### Kombinace existujících sloupců

---

Tento způsob lze použít, pokud chceš vytvořit nový sloupec pomocí operací mezi existujícími sloupci:

In [21]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni
0,Matouš,23,65,160,Holinka
1,Marek,25,70,170,Párek
2,Lukáš,27,75,180,Holinka
3,Jan,29,80,190,Novák


<br>

Nejprve výpočet BMI: `vaha [kg] / (vyska^2) [m]`

In [22]:
uzivatele_df['BMI'] = round(uzivatele_df['vaha'] / ((uzivatele_df['vyska'] / 100) ** 2), 1)

In [23]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI
0,Matouš,23,65,160,Holinka,25.4
1,Marek,25,70,170,Párek,24.2
2,Lukáš,27,75,180,Holinka,23.1
3,Jan,29,80,190,Novák,22.2


<br>

Následně přidám nový sloupeček `'Celé jméno'`:

In [24]:
uzivatele_df['Celé jméno'] = uzivatele_df['prijmeni'] + ' ' + uzivatele_df['jmeno']

In [25]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,Celé jméno
0,Matouš,23,65,160,Holinka,25.4,Holinka Matouš
1,Marek,25,70,170,Párek,24.2,Párek Marek
2,Lukáš,27,75,180,Holinka,23.1,Holinka Lukáš
3,Jan,29,80,190,Novák,22.2,Novák Jan


#### Metoda `apply`

---

Pokud potřebuješ aplikovat uživ. funkci **pro stávající sloupec**, využiješ metodu `apply`:

In [26]:
def vytvor_email(prijmeni, domena="gmail.com"):
    return f"{prijmeni.lower()}@{domena}"

In [27]:
vytvor_email(prijmeni='holinka')

'holinka@gmail.com'

<br>

Pokud použiju **jeden parametr**, musím aplikovat **jeden sloupec**:

In [28]:
uzivatele_df['E-mail'] = uzivatele_df['prijmeni'].apply(vytvor_email)

In [29]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,Celé jméno,E-mail
0,Matouš,23,65,160,Holinka,25.4,Holinka Matouš,holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,Párek Marek,párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,Holinka Lukáš,holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,Novák Jan,novák@gmail.com


<br>

Pokud budeš potřebovat pracovat **s více sloupečky**:

In [30]:
from pandas import Series

In [31]:
# from unidecode import unidecode

In [32]:
def vytvor_novy_email(radek, domena="gmail.com"):
    se_specialnimi_znaky = f"{radek['jmeno'].lower()}.{radek['prijmeni'].lower()}@{domena}"
    # return unidecode(se_specialnimi_znaky)
    return se_specialnimi_znaky

In [33]:
jeden_uzivatel = DataFrame({'jmeno': ['Matous'], 'prijmeni': ['Test']})

In [34]:
jeden_uzivatel.iloc[0]

jmeno       Matous
prijmeni      Test
Name: 0, dtype: object

In [35]:
vytvor_novy_email(radek=jeden_uzivatel.iloc[0])

'matous.test@gmail.com'

In [36]:
# vytvor_novy_email(radek=['matous', 'test'])  # TypeError

In [37]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,Celé jméno,E-mail
0,Matouš,23,65,160,Holinka,25.4,Holinka Matouš,holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,Párek Marek,párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,Holinka Lukáš,holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,Novák Jan,novák@gmail.com


In [38]:
# DataFrame.apply?

In [39]:
# uzivatele_df['Nový e-mail'] = uzivatele_df.apply(vytvor_novy_email)  # KeyError

In [40]:
uzivatele_df['Nový e-mail'] = uzivatele_df.apply(vytvor_novy_email, axis=1)

In [41]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,Celé jméno,E-mail,Nový e-mail
0,Matouš,23,65,160,Holinka,25.4,Holinka Matouš,holinka@gmail.com,matouš.holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,Párek Marek,párek@gmail.com,marek.párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,Holinka Lukáš,holinka@gmail.com,lukáš.holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,Novák Jan,novák@gmail.com,jan.novák@gmail.com


<br>

Nepovinný argument `axis` v ohlášení říká, že chce aplikovat metodu **na řádkové hodnoty**.

Tento způsob lze použít, pokud chceš vytvořit **nový sloupec** pomocí nějaké funkce na jednotlivé řádky nebo sloupce `DataFrame`.

<br>

### Odstraňování sloupečků

---

K odstranění sloupců z `DataFrame` objektu můžeš použít několik metod:
1. pythonovskou funkci `del`,
2. metodu `drop`,
3. metodu `pop`.

In [42]:
from pandas import DataFrame

In [43]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,Celé jméno,E-mail,Nový e-mail
0,Matouš,23,65,160,Holinka,25.4,Holinka Matouš,holinka@gmail.com,matouš.holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,Párek Marek,párek@gmail.com,marek.párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,Holinka Lukáš,holinka@gmail.com,lukáš.holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,Novák Jan,novák@gmail.com,jan.novák@gmail.com


<br>

Pokud odstraňuješ sloupec/ řádek, **který neexistuje**, dostaneš chybu:

In [44]:
# uzivatele_df.drop('dalsi_email')  # KeyError: axis == 0

In [45]:
'dalsi_email' in uzivatele_df.index

False

In [46]:
'dalsi_email' in uzivatele_df.columns

False

In [47]:
uzivatele_df.drop('dalsi_email', errors='ignore', axis=1)

,jmeno,vek,vaha,vyska,prijmeni,BMI,Celé jméno,E-mail,Nový e-mail
0,Matouš,23,65,160,Holinka,25.4,Holinka Matouš,holinka@gmail.com,matouš.holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,Párek Marek,párek@gmail.com,marek.párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,Holinka Lukáš,holinka@gmail.com,lukáš.holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,Novák Jan,novák@gmail.com,jan.novák@gmail.com


In [48]:
uzivatele_df.drop('Nový e-mail', axis=1, inplace=True)

In [49]:
uzivatele_df.drop(2)  # axis=0 --> odstraň hodnotu indexu

,jmeno,vek,vaha,vyska,prijmeni,BMI,Celé jméno,E-mail
0,Matouš,23,65,160,Holinka,25.4,Holinka Matouš,holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,Párek Marek,párek@gmail.com
3,Jan,29,80,190,Novák,22.2,Novák Jan,novák@gmail.com


In [50]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,Celé jméno,E-mail
0,Matouš,23,65,160,Holinka,25.4,Holinka Matouš,holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,Párek Marek,párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,Holinka Lukáš,holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,Novák Jan,novák@gmail.com


<br>

**🧠 CVIČENÍ 🧠, zpracuj zadané body v úloze.**

Máš DataFrame `zamestnanci` s informacemi o zaměstnancích tvé firmy.

Potřebuješ následující:
1. **přidat sloupec** s názvem `plat_na_hodinu`, který bude vypočítán jako průměr platu zaměstnanců dělený počtem odpracovaných hodin.
2. Sloupec `plat_na_hodinu` zaokrouhli **na celé číslo**,
3. **přejmenuj sloupec** `jmeno` na `cele_jmeno`,
4. **odstraň sloupec** `id`.
5. ☢️ **vytvoř funkci** `preved_datum`, která změní formát data ve sloupci `datum_narozeni` z `YYYY-MM-DD` na `DD/MM/YYYY`,

In [51]:
from datetime import datetime

import pandas as pd

In [52]:
zamestnanci_df = pd.DataFrame({'id': [1, 2, 3, 4],
                            'jmeno': ['Jan Novák', 'Petr Soukup', 'Marie Horáková', 'Jana Svobodová'],
                            'datum_narozeni': ['1990-05-15', '1985-12-10', '1992-07-25', '1988-03-20'],
                            'plat': [40_000, 45_000, 35_000, 50_000],
                            'odprac_hodiny': [160, 180, 150, 200]})

In [53]:
zamestnanci_df

,id,jmeno,datum_narozeni,plat,odprac_hodiny
0,1,Jan Novák,1990-05-15,40000,160
1,2,Petr Soukup,1985-12-10,45000,180
2,3,Marie Horáková,1992-07-25,35000,150
3,4,Jana Svobodová,1988-03-20,50000,200


In [54]:
zamestnanci_df['Plat na hodinu'] = (zamestnanci_df['plat'] / zamestnanci_df['odprac_hodiny']).astype('int64')

In [55]:
zamestnanci_df = zamestnanci_df.rename(columns={'jmeno': 'Celé jméno'})

In [56]:
zamestnanci_df.drop(columns=['id'])

,Celé jméno,datum_narozeni,plat,odprac_hodiny,Plat na hodinu
0,Jan Novák,1990-05-15,40000,160,250
1,Petr Soukup,1985-12-10,45000,180,250
2,Marie Horáková,1992-07-25,35000,150,233
3,Jana Svobodová,1988-03-20,50000,200,250


In [57]:
# '1990-05-15' --> '15/05/1990'
zamestnanci_df.loc[0, 'datum_narozeni']

'1990-05-15'

In [58]:
# str / datetime64
type(zamestnanci_df.loc[0, 'datum_narozeni'])

str

In [59]:
prevedene_datum = datetime.strptime(zamestnanci_df.loc[0, 'datum_narozeni'], '%Y-%m-%d')

In [60]:
prevedene_datum

datetime.datetime(1990, 5, 15, 0, 0)

In [61]:
hotove_datum = prevedene_datum.strftime('%d/%m/%Y')

In [62]:
hotove_datum

'15/05/1990'

In [63]:
def prevod_datumu(datum,
                  puvodni_format='%Y-%m-%d',
                  novy_format='%d/%m/%Y'):
    prevedene_datum = datetime.strptime(datum, puvodni_format)
    return prevedene_datum.strftime(novy_format)

In [64]:
datumy_sloupecek = Series(('1990-05-15', '2001-02-28'))

In [65]:
datumy_sloupecek[0]

'1990-05-15'

In [66]:
prevod_datumu(datumy_sloupecek[1])

'28/02/2001'

In [67]:
zamestnanci_df['Převedené datum'] = zamestnanci_df['datum_narozeni'].apply(prevod_datumu)

In [68]:
zamestnanci_df

,id,Celé jméno,datum_narozeni,plat,odprac_hodiny,Plat na hodinu,Převedené datum
0,1,Jan Novák,1990-05-15,40000,160,250,15/05/1990
1,2,Petr Soukup,1985-12-10,45000,180,250,10/12/1985
2,3,Marie Horáková,1992-07-25,35000,150,233,25/07/1992
3,4,Jana Svobodová,1988-03-20,50000,200,250,20/03/1988


<details>
    <summary>▶️ Řešení</summary>
    
```python
zamestnanci['plat_na_hodinu'] = round(zamestnanci['plat'] / zamestnanci['odprac_hodiny'])

zamestnanci = zamestnanci.rename(columns={'jmeno': 'cele_jmeno'})

zamestnanci = zamestnanci.drop(columns=['id'])


def preved_datum(datum_narozeni, vstupni_format='%Y-%m-%d', vystupni_format='%d/%m/%Y'):
    dt = datetime.strptime(datum_narozeni, vstupni_format)
    prevedene_datum = dt.strftime(vystupni_format)
    return prevedene_datum


# zamestnanci['datum_narozeni'] = pd.to_datetime(zamestnanci['datum_narozeni']).dt.strftime('%d/%m/%Y')  #  I. řešení
# zamestnanci['datum_narozeni'] = zamestnanci['datum_narozeni'].apply(preved_datum)                      # II. řešení
```
</details>

---



<img src="https://imgs.search.brave.com/avW2sADDSquCer8A7CT2t7rtq5ea-z8nvmRLilPc4-U/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9kZ2Vp/dTNmejI4Mng1LmNs/b3VkZnJvbnQubmV0/L2cvbC9sZ2ZwMTgz/OStqb2luLXVzLW9y/LWRpZS1kYXJ0aC12/YWRlci1zdGFyLXdh/cnMtMzB0aC1hbm5p/bnZlcnNhcnktcG9z/dGVyLmpwZw" width="160" style="margin-left:auto; margin-right:auto"/>

<br>

## Spojování dat

---

Takovými operacemi si můžeš představit jednoduché operace jako **spojování** (*konkatenace*) **dvou a více datasetů**, tedy tabulek.

Obdoba pro složitější *JOINY* jako u databází a jejich tabulek.

`pandas` obsahuje různé *funkce* a *metody*, které umožňují tento způsob práce:

* ✅ funkce `concat`,
* ✅ funkce `merge`,
* ⛔️ metoda `append` (*outdated*),
* ✅ metoda `join`.

### Spojování s CONCAT

---

Funkce `concat()` v `pandas` umožňuje spojit **dva nebo více**:
1. **sloupečků** tabulky,
    - *horizontálně*, po sloupcích,
    - *vertikálně*, po Indexech,
3. **tabulek** samotných,
    - *horizontálně*, po sloupcích,
    - *vertikálně*, po Indexech,

<br>

Použití funkce `concat()` spočívá **v předání seznamu objektů**, které chceš spojit, a parametru `axis`, který určuje osu, podle které se májí objekty spojit.

#### Spojení sloupečků, CONCAT

---

In [69]:
from pandas import concat, Series, DataFrame

In [70]:
sloupecek_1 = Series(['A', 'B', 'C'], index=[1, 2, 3])
sloupecek_2 = Series(['D', 'E', 'F'], index=[4, 5, 6])

In [71]:
sloupecek_1

1    A
2    B
3    C
dtype: object

In [72]:
sloupecek_2

4    D
5    E
6    F
dtype: object

<br>

##### Spojení **vertikální** (Indexy navazují, tabulky "pod sebou"):

---

In [73]:
# spojene_sloupce_vertikalne = concat(sloupecek_1, sloupecek_2, axis=0)

<br>

Nesprávné spojení, potřeba nahlédnout do popisku funkce:

In [74]:
# concat?

In [75]:
spojene_sloupce_vertikalne = concat((sloupecek_1, sloupecek_2), axis=0)

Nyní spojujeme sekvenci `tuple`, což už spadá mezi iterovatelné datové typy.
<br>

Jaký parametr je `axis` a jakých může nabývat hodnot:

In [76]:
# concat?

In [77]:
spojene_sloupce_vertikalne

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

<br>

##### Spojení **horizontální** (vedle sebe, přidání sloupečku):

---

In [78]:
sloupce_spojene_horizontalne = concat([sloupecek_1, sloupecek_2], axis=1)

In [79]:
sloupce_spojene_horizontalne

,0,1
1,A,NaN
2,B,NaN
3,C,NaN
4,NaN,D
5,NaN,E
6,NaN,F


<br>

#### Spojení tabulek, CONCAT

---

In [80]:
uzivatele_tab_1 = {'jmeno': ['Matouš', 'Marek', 'Lukáš'],
                   'vek': [25, 30, 35]}

In [81]:
uzivatele_tab_2 = {'jmeno': ['Petr', 'Jan', 'Michal'],
                   'vek': [40, 45, 50]}

In [82]:
uzivatele_tab_1_df = DataFrame(uzivatele_tab_1)
uzivatele_tab_2_df = DataFrame(uzivatele_tab_2)

In [83]:
uzivatele_tab_1_df

,jmeno,vek
0,Matouš,25
1,Marek,30
2,Lukáš,35


In [84]:
uzivatele_tab_2_df

,jmeno,vek
0,Petr,40
1,Jan,45
2,Michal,50



#### Jak zapsat spojení 2 různých tabulek?

---

Stejně jako u sloupečků, nezapomínat **na poziční argumenty** (závorka).

Stejně tak, vybrat správnou osu.

In [85]:
spojene_tab_df = concat([uzivatele_tab_1_df, uzivatele_tab_2_df], axis=0)

In [86]:
spojene_tab_df

,jmeno,vek
0,Matouš,25
1,Marek,30
2,Lukáš,35
0,Petr,40
1,Jan,45
2,Michal,50


#### Co s duplicitními Indexy?

---

Tady se ovšem **zduplikovali hodnoty** některých indexů.

Ty je potřeba opravit tímto postupem:
1. **Vytvořím nový sloupeček** pro Indexy,
2. **odstraním starý sloupeček** s duplicitami.

In [87]:
spojene_tab_df = spojene_tab_df.reset_index()  # Přidá nový Index

<br>

Metoda `reset_index` přidá nový, defaultní sloupec s pomocným Indexem.

In [88]:
spojene_tab_df

,index,jmeno,vek
0,0,Matouš,25
1,1,Marek,30
2,2,Lukáš,35
3,0,Petr,40
4,1,Jan,45
5,2,Michal,50


In [89]:
spojene_tab_df = spojene_tab_df.drop("index", axis=1)

In [90]:
spojene_tab_df

,jmeno,vek
0,Matouš,25
1,Marek,30
2,Lukáš,35
3,Petr,40
4,Jan,45
5,Michal,50


<br>

Kontrola nového rozsahu **pro Index**:

In [91]:
spojene_tab_df.index

RangeIndex(start=0, stop=6, step=1)

<br>

Pozůstatky sloupečků:

In [92]:
spojene_tab_df.columns

Index(['jmeno', 'vek'], dtype='object')

<br>

Nastavení jména nového Indexu:

In [93]:
spojene_tab_df.index.names

FrozenList([None])

<br>

Doplníš jméno **pro Index**:

In [94]:
spojene_tab_df.index.names = ["Nový Index"]

In [95]:
spojene_tab_df.index.names

FrozenList(['Nový Index'])

In [96]:
spojene_tab_df.index

RangeIndex(start=0, stop=6, step=1, name='Nový Index')

In [97]:
spojene_tab_df

,jmeno,vek
Nový Index,,
0,Matouš,25
1,Marek,30
2,Lukáš,35
3,Petr,40
4,Jan,45
5,Michal,50


Pro odchytávání **duplicitních indexů** můžeš doplnit parametr `verify_integrity=True`, případně pokud je irelevantní, ignorovat jej úplně `ignore_index=True`.

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí CONCAT**

In [98]:
from pandas import DataFrame, concat, Series

In [99]:
data_populace = {"City": ["New York", "London", "Tokyo"],
                 "Population": [8.4, 9.0, 13.9]}

data_plocha = {"City": ["New York", "London", "Tokyo"],
               "Area": [783, 1572, 2194]}

data_kontinent = {"City": ["New York", "London", "Tokyo"],
                  "Continent": ["North America", "Europe", "Asia"]}

# ❓ Vytvoř funkci "spoj_data_mest", která spojí 3 předchozí tabulky❓

In [100]:
data_populace_df = DataFrame(data_populace)
data_plocha_df = DataFrame(data_plocha)
data_kontinent_df = DataFrame(data_kontinent)

In [101]:
data_populace_df

,City,Population
0,New York,8.4
1,London,9.0
2,Tokyo,13.9


In [102]:
data_plocha_df

,City,Area
0,New York,783
1,London,1572
2,Tokyo,2194


In [103]:
data_kontinent_df

,City,Continent
0,New York,North America
1,London,Europe
2,Tokyo,Asia


In [104]:
data_populace_df = DataFrame(data_populace)

In [105]:
data_populace_df.set_index('City')

,Population
City,
New York,8.4
London,9.0
Tokyo,13.9


In [106]:
def spoj_data_pro_mesta(*args):
    sekvenci_udaje = list()

    for arg in args:
        sekvenci_udaje.append(arg.set_index('City'))

    return concat(sekvenci_udaje, axis=1)

In [107]:
spojene_data = spoj_data_pro_mesta(data_populace_df,
                    data_plocha_df,
                    data_kontinent_df)

In [108]:
spojene_data

,Population,Area,Continent
City,,,
New York,8.4,783,North America
London,9.0,1572,Europe
Tokyo,13.9,2194,Asia


<details>
    <summary>▶️ Řešení</summary>
    
```python
from pandas import DataFrame, concat

data_populace = {
    "City": ["New York", "London", "Tokyo"],
    "Population": [8.4, 9.0, 13.9],
}

data_plocha = {
    "City": ["New York", "London", "Tokyo"],
    "Area": [783, 1572, 2194],
}

data_kontinent = {
    "City": ["New York", "London", "Tokyo"],
    "Continent": ["North America", "Europe", "Asia"],
}

populace_df = DataFrame(data_populace)
plocha_df = DataFrame(data_plocha)
kontinent_df = DataFrame(data_kontinent)


populace_df = DataFrame(data_populace)
plocha_df = DataFrame(data_plocha)
kontinent_df = DataFrame(data_kontinent)


def spoj_data_mest(*args) -> DataFrame:
    data = [arg.set_index('City') for arg in args]
    return concat(data, axis=1)


result = spoj_data_mest(populace_df, plocha_df, kontinent_df)
print(result)
```
</details>

<img src="https://imgs.search.brave.com/wR9sz6BPwR_LCDm6Esd_B9VgUBSbW4wbTPi2T9DrdBc/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly90NC5m/dGNkbi5uZXQvanBn/LzA4LzQwLzM1LzIx/LzM2MF9GXzg0MDM1/MjE1Ml84N1N2T0kx/REM2RElCaHkxc0xz/VnpnYmNIUk1UZno0/cC5qcGc" width="230" style="margin-left:auto; margin-right:auto"/>

### CONCAT a volitelný parametr JOIN

---

V jednoduchých ukázkách, jako jsou ty výše, stačilo **tabulky a sloupečky spojit**.

To prakticky není vždy ideální řešení, protože některé sloupečky se mohou lišit.

Třeba, pokud mají různé jména.

In [109]:
from pandas import DataFrame, concat

In [110]:
data_tab_1_df = DataFrame({"A": ["A1", "A2"],
                           "B": ["B1", "B2"],
                           "C": ["C1", "C2"]},
                          index=[1, 2])

In [111]:
data_tab_2_df = DataFrame({"B": ["B3", "B4"],
                           "C": ["C3", "C4"],
                           "D": ["D3", "D4"]},
                          index=[3, 4])

In [112]:
data_tab_1_df

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


In [113]:
data_tab_2_df

,B,C,D
3,B3,C3,D3
4,B4,C4,D4



#### Horizontální spojení dvou tabulek

---

In [114]:
vystup_tab_df = concat([data_tab_1_df, data_tab_2_df])  # axis=0

In [115]:
vystup_tab_df

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


Pokud některá data chybějí, jsou automaticky vyplněná **neznámými hodnotami**.

<br>

Řešením takové situace můžeš být zavedení jiného **způsobu spojování**.

Tedy nepoužívat defaultní argument `join='outer'`, ale `join='inner'`:

In [116]:
vystup_bez_nan_df = concat([data_tab_1_df, data_tab_2_df],
                           join='inner',
                           axis=0)

In [117]:
vystup_bez_nan_df

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


Použití `outer` odpovídá implicitnímu chování funkce `concat`:

In [118]:
vystup_vc_nan_df = concat([data_tab_1_df, data_tab_2_df],
                           join='outer',
                           axis=0)

In [119]:
vystup_vc_nan_df

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


#### 📝 Souhrn k funkci `concat`

---

| Situace                                             | Použiješ                   | join?               |
| --------------------------------------------------- | -------------------------- | ------------------- |
| Spojuješ tabulky se stejnými sloupci pod sebe       | `pd.concat([...])`         | **nepotřebuješ**    |
| Spojuješ tabulky vedle sebe podle indexu            | `pd.concat([...], axis=1)` | volitelně           |
| Máš různé indexy a chceš spojit jen průnik          | `pd.concat([...], axis=1)` | `join='inner'`      |
| Chceš **zachovat vše**, i když něco chybí           | `pd.concat([...], axis=1)` | `join='outer'`      |
| Chceš sloučit podle **konkrétního sloupce** | ❌ `concat` nestačí         | 👉 použij `merge()` |


<img src="https://imgs.search.brave.com/IZ8jkD1GaDV2tXwgwzM1Xawk-jBMejygby1KcermQp8/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly90NC5m/dGNkbi5uZXQvanBn/LzA4LzQ2LzQ3LzE1/LzM2MF9GXzg0NjQ3/MTU2MF95Y281RnhM/UG5RR1c4NW1OWHBD/UmcxUWptSzRiMnp3/eS5qcGc" width="200" style="margin-left:auto; margin-right:auto"/>



### Spojování pomocí `MERGE`

---

Další funkcí pro spojování `DataFrame` objektů je `merge`.

Tato funkce je vhodná pro spojování DataFrame objektů, které **mají společné sloupce**.

Můžeš lépe zadávat typ spojení (parametr `how='inner' | 'outer' | 'left' | 'join'`).

Dále ti umožní **definovat sloupec, nebo sloupce**, na kterých chceš spojení provést (parametr `on`).

In [120]:
from pandas import merge

In [121]:
uzivatele_tab_1 = {'jmeno': ['Alice', 'Bob', 'Charlie', 'David'],
                   'vek': [25, 30, 35, 40],
                   'mesto': ['Brno', 'Praha', 'Plzen', 'Ostrava']}

In [122]:
uzivatele_tab_2 = {'jmeno': ['Alice', 'David', 'Emma', 'Frank'],
                   'pocet_prijemcu': [100, 200, 150, 250]}

In [123]:
uzivatele_tab_1_df = DataFrame(uzivatele_tab_1)
uzivatele_tab_2_df = DataFrame(uzivatele_tab_2)

In [124]:
uzivatele_tab_1_df

,jmeno,vek,mesto
0,Alice,25,Brno
1,Bob,30,Praha
2,Charlie,35,Plzen
3,David,40,Ostrava


In [125]:
uzivatele_tab_2_df

,jmeno,pocet_prijemcu
0,Alice,100
1,David,200
2,Emma,150
3,Frank,250


#### Spojení FULL JOIN

---

Jinými slovy spoj všechny záznamy, **pro konkrétní sloupeček**:

In [126]:
mergnute_df = merge(uzivatele_tab_1_df,
                    uzivatele_tab_2_df,
                    on='jmeno',
                    how='outer')

In [127]:
mergnute_df

,jmeno,vek,mesto,pocet_prijemcu
0,Alice,25.0,Brno,100.0
1,Bob,30.0,Praha,NaN
2,Charlie,35.0,Plzen,NaN
3,David,40.0,Ostrava,200.0
4,Emma,NaN,NaN,150.0
5,Frank,NaN,NaN,250.0


#### Spojené INNER MERGE

---

In [128]:
mergnute_inner_df = merge(uzivatele_tab_1_df,
                          uzivatele_tab_2_df,
                          on='jmeno',
                          how='inner')

In [129]:
mergnute_inner_df

,jmeno,vek,mesto,pocet_prijemcu
0,Alice,25,Brno,100
1,David,40,Ostrava,200


<br>

Zde byla ale jména sloupců na první pohled odlišná.

Ne ve všech situacích to platí:

In [130]:
df8 = DataFrame({'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
                 'poradi': [1, 2, 3, 4]})

In [131]:
df9 = DataFrame({'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
                 'poradi': [3, 1, 4, 2]})

Funkce sama **doplní přípony**, aby rozlišila **mezi oběma původními sloupci**.

**Pokud potřebuješ vlastní přípony**, můžeš vyzkoušet volitelný argument pro `suffixes`:

In [132]:
df8

,jmeno,poradi
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [133]:
df9

,jmeno,poradi
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [134]:
vystup_konflikt = merge(df8, df9, on="jmeno")

In [135]:
vystup_konflikt

,jmeno,poradi_x,poradi_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


In [136]:
vystup_vlastni_pripony = merge(df8, df9, on='jmeno', suffixes=('_DF8', '_DF9'))

In [137]:
vystup_vlastni_pripony

,jmeno,poradi_DF8,poradi_DF9
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Obecně platí, že pokud potřebuješ *spojit* dva nebo více `DataFrame` objektů podle **společného sloupce nebo sloupců**, použij funkci `merge()`.

Pokud chceš jenom **přidat další řádky nebo sloupce** do existujícího `DataFrame` objektu, použij funkci `concat`.

#### Ukázka MERGE funkce s dopomocí `LEFT`

In [138]:
uzivatele_tab_1_df

,jmeno,vek,mesto
0,Alice,25,Brno
1,Bob,30,Praha
2,Charlie,35,Plzen
3,David,40,Ostrava


In [139]:
uzivatele_tab_2_df

,jmeno,pocet_prijemcu
0,Alice,100
1,David,200
2,Emma,150
3,Frank,250


In [140]:
vystup_left_join_df = merge(uzivatele_tab_1_df,
                            uzivatele_tab_2_df,
                            on='jmeno',
                            how='left')

In [141]:
vystup_left_join_df

,jmeno,vek,mesto,pocet_prijemcu
0,Alice,25,Brno,100.0
1,Bob,30,Praha,NaN
2,Charlie,35,Plzen,NaN
3,David,40,Ostrava,200.0


V ukázce výš je použitý *left join*.

Tedy ve výsledku uvidíš **celou první** (levou tabulku) a z druhé pouze ty záznamy, které mají ve spojovacím sloupci `jmeno` společnou hodnotu.

In [142]:
vystup_left_join_obr_df = merge(uzivatele_tab_2_df,
                            uzivatele_tab_1_df,
                            on='jmeno',
                            how='left')

In [143]:
uzivatele_tab_2_df

,jmeno,pocet_prijemcu
0,Alice,100
1,David,200
2,Emma,150
3,Frank,250


In [144]:
vystup_left_join_obr_df

,jmeno,pocet_prijemcu,vek,mesto
0,Alice,100,25.0,Brno
1,David,200,40.0,Ostrava
2,Emma,150,NaN,NaN
3,Frank,250,NaN,NaN


<img src="https://imgs.search.brave.com/-VgG6K4KBK8o5Oy7foYt_v7sPQTBbdERXhQ_ZqsqYLE/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9hLnBp/bmF0YWZhcm0uY29t/LzUwMHgzODQvOWZj/MDU0ZTM5Zi9qb2lu/LXVzLmpwZw" width="300" style="margin-left:auto; margin-right:auto"/>

### Spojení pomocí indexů, `JOIN`

---

Tato metoda slouží k propojení dvou `DataFrame` objektů na základě **jejich indexů nebo hodnot**.

Je velice podobná funkci `merge` ale je přímo součástí `DataFrame` objektu a je snazší ji aplikovat:

In [145]:
uzivatele_1 = {'jmeno': ['Alice', 'Bob', 'Petr'],
               'vek': [25, 30, 35]}

In [146]:
uzivatele_2 = {'jmeno': ['Alice', 'Bob', 'Petr'],
               'pocet_prijemcu': [100, 150, 200]}

In [147]:
df_uzivatele_1 = DataFrame(uzivatele_1)
df_uzivatele_2 = DataFrame(uzivatele_2)

In [148]:
df_uzivatele_1

,jmeno,vek
0,Alice,25
1,Bob,30
2,Petr,35


In [149]:
df_uzivatele_2

,jmeno,pocet_prijemcu
0,Alice,100
1,Bob,150
2,Petr,200


In [150]:
df_uzivatele_2.index.names == df_uzivatele_1.index.names == [None]

True

In [151]:
uzivatele_1_df = df_uzivatele_1.set_index('jmeno')

In [152]:
uzivatele_2_df = df_uzivatele_2.set_index('jmeno')

In [153]:
uzivatele_1_df

,vek
jmeno,
Alice,25
Bob,30
Petr,35


In [154]:
uzivatele_2_df

,pocet_prijemcu
jmeno,
Alice,100
Bob,150
Petr,200


In [155]:
vysledek_join_metody = uzivatele_1_df.join(uzivatele_2_df)

In [156]:
vysledek_join_metody

,vek,pocet_prijemcu
jmeno,,
Alice,25,100
Bob,30,150
Petr,35,200


Hlavní rozdíl mezi těmito funkcemi je způsob určení **sloupce nebo sloupců**, podle kterých se má propojení provést.

1. Metoda `join()` propojuje DataFrame objekty **na základě jejich Indexů**.
2. Funkce `merge()` umožňuje propojit `DataFrame` objekty **na základě hodnoty v jednom nebo více sloupcích**.

## 📑Souhrn ke spojování

---

| Objekt | Kdy jej použít? | Směr spojení |
| :- | :- | :- |
| `concat` | Když chceš poskládat tabulky na sebe (řádky) nebo vedle sebe (sloupce). | Vertikálně / Horizontálně | 
| `merge` |	Když chcete spojit tabulky podle společného klíče/sloupce, podobně jako SQL JOIN. | Horizontálně |
| `join` | Když chcete spojit tabulky podle společného Indexu (ne klíče/sloupce). | Horizontálně |
| `append` | (Zastaralé) Používalo se k přidání řádků, nyní používejte concat. | Vertikálně |

<br>

**🧠 CVIČENÍ 🧠, procvič si spojování**

Tvým úkolem je propojit tyto tabulky podle zadání:
1. Použij spojovací funkci pro spojení objektů typu `DataFrame` , které vytvoříš z proměnných `data_zamestnanci_1` a `data_zamestnanci_2`,
2. použij spojovací funkci, které zadáš vzniklý objekt typu `DataFrame`  z bodu 1. a druhý `DataFrame`, který vytvoříš z proměnné `data_oddeleni` (spoj přes sloupeček `"oddělení_id"`),
3. vytvoř výsledný objekt typu `DataFrame` , který obsahuje pouze zaměstnance z oddělení `IT`.

In [157]:
data_zamestnanci_1 = {
    'id': [1, 2, 3],
    'jméno': ['Jan', 'Marie', 'Petr'],
    'příjmení': ['Novák', 'Svobodová', 'Dvořák'],
    'oddělení_id': [100, 200, 100]
}

data_zamestnanci_2 = {
    'id': [4, 5],
    'jméno': ['Anna', 'Josef'],
    'příjmení': ['Kopecká', 'Vondráček'],
    'oddělení_id': [200, 100]
}

data_oddeleni = {
    'oddělení_id': [100, 200],
    'název_oddělení': ['IT', 'HR']
}

<details>
    <summary>▶️ Řešení</summary>
    
```python
df_zamestnanci_1 = pd.DataFrame(data_zamestnanci_1)
df_zamestnanci_2 = pd.DataFrame(data_zamestnanci_2)
df_oddeleni = pd.DataFrame(data_oddeleni)

df_vsichni_zamestnanci = pd.concat([df_zamestnanci_1, df_zamestnanci_2], ignore_index=True)

# df1 = pd.concat(
#     [pd.DataFrame(data) for data in [data_zamestnanci_1, data_zamestnanci_2]]
# ).reset_index(drop = True).set_index('id')

df_vsechna_oddeleni = pd.merge(df_vsichni_zamestnanci, df_oddeleni, on="oddělení_id")

df_jen_it = df_vsechna_oddeleni[df_vsechna_oddeleni["název_oddělení"] == "IT"]
```
</details>

---